In [2]:
import os
import importlib
import zipfile as zf
import pandas as pd
import numpy as np
import joblib
import cvm
import super_query

importlib.reload(cvm)
importlib.reload(super_query)
pd.options.display.max_colwidth = 15
pd.options.display.max_rows = 4

In [6]:
filenames = sorted(os.listdir('./data/raw/'))
filenames

['dfp_cia_aberta_2010.zip',
 'dfp_cia_aberta_2011.zip',
 'dfp_cia_aberta_2012.zip',
 'dfp_cia_aberta_2013.zip',
 'dfp_cia_aberta_2014.zip',
 'dfp_cia_aberta_2015.zip',
 'dfp_cia_aberta_2016.zip',
 'dfp_cia_aberta_2017.zip',
 'dfp_cia_aberta_2018.zip',
 'dfp_cia_aberta_2019.zip',
 'dfp_cia_aberta_2020.zip',
 'dfp_cia_aberta_2021.zip',
 'itr_cia_aberta_2020.zip',
 'itr_cia_aberta_2021.zip',
 'itr_cia_aberta_2022.zip']

In [9]:
# full scan = 8m 10s -> 5.6 GB df memory space
# formated colums = 4m 30s -> 3.5 GB df memory space
# zstd -> 16.5 seg
# lz4 -> 15.4 seg
# df_all = pd.DataFrame()
df = pd.DataFrame()
for parent_filename in filenames:
    print(parent_filename)
    file_path = f'./data/raw/{parent_filename}'
    parent_file = zf.ZipFile(file_path)
    # print(parent_file.namelist())
    child_filenames = parent_file.namelist()
    for child_filename in child_filenames[1:]:
        # print(child_parent_file_name)
        child_file = parent_file.open(child_filename)
        df_child = pd.read_csv(child_file, **kwargs)
        df_child = cvm.process_child_dataframe(df_child)        
        df = pd.concat([df, df_child], ignore_index=True)

sort_by = [
    'CD_CVM', 'GRUPO_DFP', 'VERSAO', 'ORDEM_EXERC', 'DT_REFER', 'CD_CONTA'
]
df.sort_values(by=sort_by, ignore_index=True, inplace=True)
print('Dataset sorted')
columns_category = df.columns[0:-1]
df[columns_category] = df[columns_category].astype('category')

full_path = './data/processed/dataset.pkl'
with open(full_path, 'wb') as f:
    joblib.dump(df, f, compress='lz4')
print('Dataset saved')

dfp_cia_aberta_2010.zip
dfp_cia_aberta_2011.zip
dfp_cia_aberta_2012.zip
dfp_cia_aberta_2013.zip
dfp_cia_aberta_2014.zip
dfp_cia_aberta_2015.zip
dfp_cia_aberta_2016.zip
dfp_cia_aberta_2017.zip
dfp_cia_aberta_2018.zip
dfp_cia_aberta_2019.zip
dfp_cia_aberta_2020.zip
dfp_cia_aberta_2021.zip
itr_cia_aberta_2020.zip
itr_cia_aberta_2021.zip
itr_cia_aberta_2022.zip
Dataset sorted
Dataset saved


In [3]:
df = super_query.update_dataset()
df

In [11]:
with open('./data/processed/dataset.pkl', 'rb') as f:
    df = joblib.load(f)
df

,CD_CVM,CNPJ_CIA,DENOM_CIA,GRUPO_DFP,VERSAO,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,ORDEM_EXERC,CD_CONTA,DS_CONTA,ST_CONTA_FIXA,COLUNA_DF,VL_CONTA
0,94,92.693.019/...,PANATLANTIC...,DF Consolid...,1,2011-12-31,NaT,2010-12-31,-1,1,Ativo Total,True,NaN,2.825930e+08
1,94,92.693.019/...,PANATLANTIC...,DF Consolid...,1,2011-12-31,NaT,2010-12-31,-1,1.01,Ativo Circu...,True,NaN,2.193760e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17608709,518700,70.216.429/...,BRAMEX - BR...,DF Individu...,1,2013-12-31,2013-01-01,2013-12-31,0,3.99.01.03,PNB,False,NaN,2.148900e-02
17608710,518700,70.216.429/...,BRAMEX - BR...,DF Individu...,1,2013-12-31,2013-01-01,2013-12-31,0,3.99.02,Lucro Diluí...,False,NaN,0.000000e+00


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17608711 entries, 0 to 17608710
Data columns (total 14 columns):
 #   Column         Dtype   
---  ------         -----   
 0   CD_CVM         category
 1   CNPJ_CIA       category
 2   DENOM_CIA      category
 3   GRUPO_DFP      category
 4   VERSAO         category
 5   DT_REFER       category
 6   DT_INI_EXERC   category
 7   DT_FIM_EXERC   category
 8   ORDEM_EXERC    category
 9   CD_CONTA       category
 10  DS_CONTA       category
 11  ST_CONTA_FIXA  category
 12  COLUNA_DF      category
 13  VL_CONTA       float64 
dtypes: category(13), float64(1)
memory usage: 489.9 MB


In [11]:
columns_category = [
    'CD_CVM', 'CNPJ_CIA', 'DENOM_CIA', 'GRUPO_DFP', 'DT_REFER', 'DT_INI_EXERC',
    'DT_FIM_EXERC', 'CD_CONTA', 'DS_CONTA', 'COLUNA_DF']
df[columns_category] = df[columns_category].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17608711 entries, 0 to 17608710
Data columns (total 14 columns):
 #   Column         Dtype   
---  ------         -----   
 0   CD_CVM         category
 1   CNPJ_CIA       category
 2   DENOM_CIA      category
 3   GRUPO_DFP      category
 4   VERSAO         int8    
 5   DT_REFER       category
 6   DT_INI_EXERC   category
 7   DT_FIM_EXERC   category
 8   ORDEM_EXERC    int8    
 9   CD_CONTA       category
 10  DS_CONTA       category
 11  ST_CONTA_FIXA  bool    
 12  COLUNA_DF      category
 13  VL_CONTA       float64 
dtypes: bool(1), category(10), float64(1), int8(2)
memory usage: 489.9 MB
